In [3]:
import numpy as np  
#import matplotlib.pyplot as plt
np.random.seed(0)
a = np.random.randint(1, 10, size=(3, 3))
print(a)

[[6 1 4]
 [4 8 4]
 [6 3 5]]


In [12]:
print('hi')

def test(x,y,z):
    if x < z:
        return x, y
    return z
import torch
    
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # Define the layers of the model
        layers = [
        torch.nn.Linear(10, 20),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.2),
        torch.nn.Linear(20, 5),
        torch.nn.ReLU()
    ]
        self.hidden_layers = torch.nn.Sequential(*layers)
        
class My2Model(torch.nn.Module):
    def __init__(self):
        super(My2Model, self).__init__()
        # Define the layers of the model
        
        self.fc1 = torch.nn.Linear(10, 20)
        self.relu1 = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.2)
        self.fc2 = torch.nn.Linear(20, 5)
        self.relu2 = torch.nn.ReLU()

        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        return x

hi


In [13]:
print(test(5,2,3))
model = MyModel()
print(model)
model2 = My2Model()
print(model2)

3
MyModel(
  (hidden_layers): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=20, out_features=5, bias=True)
    (4): ReLU()
  )
)
My2Model(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (relu1): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=20, out_features=5, bias=True)
  (relu2): ReLU()
)


In [1]:
split_ratio = (0.6, 0.2, 0.2)
print(split_ratio[1])

0.2


In [ ]:
import os

if os.environ['COMPUTERNAME'] == 'FYNN':            # name of surface PC
    path = r"C:\Users\Surface\Masterarbeit\data\Produktionsdaten\WZ_2_Feature_Engineered_Fynn6.xlsx"
elif os.environ['COMPUTERNAME'] == 'DEIN_DESKTOPNAME':  # replace with your desktop name
    
else:
    raise ValueError("Unbekannter Computername: " + os.environ['COMPUTERNAME'])
    
print(path)

C:\Users\Surface\Masterarbeit\data\Produktionsdaten\WZ_2_Feature_Engineered_Fynn6.xlsx


In [3]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

print(type(train_x), train_x.shape)

# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

training_iter = 10


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    print(type(output))#, output.shape)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

<class 'torch.Tensor'> torch.Size([100])
<class 'gpytorch.distributions.multivariate_normal.MultivariateNormal'>
Iter 1/10 - Loss: 0.921   lengthscale: 0.693   noise: 0.693
<class 'gpytorch.distributions.multivariate_normal.MultivariateNormal'>
Iter 2/10 - Loss: 0.888   lengthscale: 0.644   noise: 0.644
<class 'gpytorch.distributions.multivariate_normal.MultivariateNormal'>
Iter 3/10 - Loss: 0.853   lengthscale: 0.598   noise: 0.598
<class 'gpytorch.distributions.multivariate_normal.MultivariateNormal'>
Iter 4/10 - Loss: 0.815   lengthscale: 0.555   noise: 0.554
<class 'gpytorch.distributions.multivariate_normal.MultivariateNormal'>
Iter 5/10 - Loss: 0.772   lengthscale: 0.514   noise: 0.513
<class 'gpytorch.distributions.multivariate_normal.MultivariateNormal'>
Iter 6/10 - Loss: 0.725   lengthscale: 0.475   noise: 0.474
<class 'gpytorch.distributions.multivariate_normal.MultivariateNormal'>
Iter 7/10 - Loss: 0.674   lengthscale: 0.438   noise: 0.437
<class 'gpytorch.distributions.mult